In [1]:
%pip install --quiet langchain pypdf pymongo langchain-openai tiktoken google-cloud-secret-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

%pip show langchain

from platform import python_version
print(python_version())

Name: langchain
Version: 0.1.1
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
3.10.12


In [3]:
import os
from google.cloud import secretmanager
from google.colab import auth
from google.colab import drive

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
def load_secrets(secrets_name, project_id):
  # Build a client
  auth.authenticate_user()
  client = secretmanager.SecretManagerServiceClient()
  secret_name = secrets_name
  # Create path to latest secret
  resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"
  # Get your secret :
  response = client.access_secret_version(request={"name": resource_name})
  secret_string = response.payload.data.decode('UTF-8')
  return secret_string

In [6]:
project_id = 'botchagalupep1'
openai_api_key = load_secrets("openai_api_key",project_id)
os.environ['OPENAI_API_KEY'] = openai_api_key
MONGODB_ATLAS_CLUSTER_URI = load_secrets("mdb_uri",project_id)
langsmith_api_key = load_secrets("langsmith_api_key",project_id)

In [7]:
# connect notebook to langsmith
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

# This key is sourced from vars.env
os.environ['LANGCHAIN_API_KEY'] = langsmith_api_key  # Uncomment and replace  with your actual API key

os.environ['LANGCHAIN_PROJECT'] = 'opencontext-1'

# To verify, you can print the variables
print(os.environ.get('LANGCHAIN_TRACING_V2'))
print(os.environ.get('LANGCHAIN_ENDPOINT'))
#print(os.environ.get('LANGCHAIN_API_KEY'))  # Uncomment if you want to print your API key (be careful with sharing your notebook)
print(os.environ.get('LANGCHAIN_PROJECT'))

true
https://api.smith.langchain.com
opencontext-1


In [8]:
from pymongo import MongoClient

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "langchain_db"
COLLECTION_NAME = "test"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "opencontext-2"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

In [9]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
    MONGODB_ATLAS_CLUSTER_URI,
    DB_NAME + "." + COLLECTION_NAME,
    OpenAIEmbeddings(disallowed_special=()),
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
)

#Pre-filtering with Similarity Search
Atlas Vector Search supports pre-filtering using MQL Operators for filtering. Below is an example index and query on the same data loaded above that allows you do metadata filtering on the “page” field. You can update your existing index with the filter defined and do pre-filtering with vector search.

Use:

~~~
{
  "name": "index_name",
  "type": "vectorSearch",
  "fields":[
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    },
    {
      "type": "filter",
      "path": "page"
    }
  ]
}
~~~


In [ ]:
#Don't run withput adding the filers in in the index JSON
#query = "What were the compute requirements for training GPT 4"
query = "What are the code components in this repositoy"

results = vector_search.similarity_search_with_score(
    query=query, k=5, pre_filter={"page": {"$eq": 1}}
)

# Display results
for result in results:
    print(result)

#Similarity Search with Score

In [19]:
#query = "Who is Edwards Deming?"
query = "What are the code components in this repositoy"

results = vector_search.similarity_search_with_score(
    query=query,
    k=5,
)

# Display results
for result in results:
    print(result)

(Document(page_content='icon:"github"}]tags:["typescript"]uid:"64c2b4af-37fc-4c75-924b-e66d2e2ad34b"etag:"21ae3f170f75c0c6660486bcfab7e85c3bc71f0e"}apiVersion:"opencontext.com/v1alpha1"kind:"CodeComponent"spec:{"type":"repository"', metadata={'_id': ObjectId('65ab042593b430af3a584743'), 'embedding': [0.011937408033999897, 0.014769199622602692, -0.007370886551755859, -0.01875976327261368, -0.016689058752938656, -0.0034060350887696726, -0.024656471396431255, 0.0024975305961432393, -0.00955129714779478, -0.020268222753939847, 0.032061640864220184, 0.004875070054848144, -0.01936314669261511, -0.022544626632458473, -0.010634646048872145, -0.01583883572979587, 0.019157447333771508, -0.043169395065903574, 0.018293511516744528, -0.02992237051247902, -0.013240169087805054, 0.020775614754106757, -0.028935013962558768, 0.002430678397651328, -0.02587695475560868, 0.0050019180548898715, 0.015125744370785354, -0.030854874024217327, -0.0027323705733133395, -0.020350503242535362, 0.03258274565827128, 

#Question Answering

In [25]:
qa_retriever = vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20},
)

In [26]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [33]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)


qa = RetrievalQA.from_chain_type(
    #llm = OpenAI(),
    llm,
    chain_type="stuff",
    retriever=qa_retriever,
    return_source_documents="True",
    chain_type_kwargs={"prompt": PROMPT},
)

#docs = qa({"query": "Explain Dr. Deming's work"})
#docs = qa({"query": "What are the code components in this repositoy"})
#docs = qa({"query": "What code components are javascript?"})
docs = qa({"query": "What code components have the tag of javascript?"})
#docs = qa({"query": "Who is the owner of crates-erp?"})

print(docs["result"])
print(docs["source_documents"])


Based on the given context, the code components that have the tag of javascript are:

1. CodeComponent with uid "64c2b4af-37fc-4c75-924b-e66d2e2ad34b" and etag "21ae3f170f75c0c6660486bcfab7e85c3bc71f0e"
2. CodeComponent with uid "bd419d60-cbf6-4f86-b523-99b602b00cc1" and etag "247e45e926da3a6072ea14b3e679f0afbf7629de"
3. CodeComponent with uid "f8597cfe-e293-44a8-8c26-02dd0e5f0f85" and etag "68fa5229ef81b8aca82f8de53d27fa97bc94e5a4"
4. CodeComponent with uid "6d5dd085-63e9-48c2-87aa-2af3995c1862" and etag "1feba2f5b48ddec7dc2d22e94e34c940467f5cf3"
5. CodeComponent with uid "9e1499e7-b5bf-4bd1-a585-753ce2e39845" and etag "556db51db46dadaa71dcc917bd77ead197c25a1a"
6. CodeComponent with uid "154aad85-981b-4ccd-bed4-1e3361d35228" and etag "61027d007d7cb4603e2439488620824c9f5a0f97"
7. CodeComponent with uid "556db51db46dadaa71dcc917bd77ead197c25a1a" and etag "556db51db46dadaa71dcc917bd77ead197c25a1a"
8. CodeComponent with uid "f8597cfe-e293-44a8-8c26-02dd0e5f0f85" and etag "68fa5229ef81b8ac

In [23]:
query = "WWhat are the code components in this repository?"

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_search.as_retriever(), return_source_documents=True
)
answer = qa_chain({"query": query})

print(answer)

{'query': 'WWhat are the code components in this repository?', 'result': 'The code components in this repository are:\n\n1. crates-erpscatterly-gl\n2. crates-erpscatter-ly\n3. crates-frontendscatter-ly\n4. crates-frontendscatterly-gl\n5. crates-templatescatter-ly\n6. dumpster-uiscatter-ly\n7. dumpster-uiscatterly-gl\n8. opencontextscatter-ly\n9. opencontextscatterly-gl\n10. platform-componentsscatter-ly\n11. platform-componentsscatterly-gl\n12. publictestscatterly-gl\n13. publictestscatter-ly\n14. sandbox.websitescatterly-gl\n15. barrow-frontendscatterly-gl\n16. barrow-frontendscatter-ly\n17. bb-barrow-fe-testscatter-ly\n18. bb-boxes-fe-testscatter-ly\n19. bb-sandbox-testscatter-ly\n20. crates-erpscatterly-gl', 'source_documents': [Document(page_content='icon:"github"}]tags:["typescript"]uid:"64c2b4af-37fc-4c75-924b-e66d2e2ad34b"etag:"21ae3f170f75c0c6660486bcfab7e85c3bc71f0e"}apiVersion:"opencontext.com/v1alpha1"kind:"CodeComponent"spec:{"type":"repository"', metadata={'_id': ObjectId(